In [ ]:
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import davies_bouldin_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
%run dataInfo.ipynb
%run RandomForest.ipynb

cleanTest = test_df
cleanTrain = train_df #doesn't have asd/class

# Drop train class column
cleanTrain = cleanTrain.drop(columns=['Class/ASD'])
cleanTest = cleanTest.drop(columns=['Class/ASD'])
cleanTrain = cleanTrain.drop(columns=['country_of_res'])
cleanTest = cleanTest.drop(columns=['country_of_res'])
cleanTrain = cleanTrain.drop(columns=['relation'])
cleanTest = cleanTest.drop(columns=['relation'])

# Combine train and test data for preprocessing
combined_df = pd.concat([cleanTrain, cleanTest], axis=0)
combined = combined_df.dropna()

#Engineer Composite Features

# Create a new feature 'mean_A_score' by averaging the scores of A1 to A10
score_cols = [f"A{i}_Score" for i in range(1, 11)]
combined['mean_A_score'] = combined[score_cols].mean(axis=1)

# score times age
combined['age_x_mean_A'] = combined['age'] * combined['mean_A_score']

# Polynomial features (squared terms)
combined['age_squared'] = combined['age'] ** 2
combined['mean_A_score_squared'] = combined['mean_A_score'] ** 2

#dont use scaler bc it makes it worse

# run the hierarchical clustering
model = AgglomerativeClustering(n_clusters=2, linkage='ward')  # Choose the number of clusters you want (e.g., 3)
combined['hierarchical_cluster'] = model.fit_predict(combined)
hierarchical_siloutte = silhouette_score(combined, combined['hierarchical_cluster'])
print(f"Silhouette Score for Hierarchical Clustering: {hierarchical_siloutte}")
db_hierarchical = davies_bouldin_score(combined, combined['hierarchical_cluster'])
print(f"Davies-Bouldin Score for Hierarchical Clustering: {db_hierarchical}")

combined['hierarchical_cluster'] = combined['hierarchical_cluster']

cluster_summary = combined.groupby('hierarchical_cluster').mean()

#Print cluster summaries
print("Hierarchical Cluster Summary:")
print(cluster_summary)

# Reduce dimensionality
pca = PCA(n_components=2)
reduced_data = pca.fit_transform(combined.drop(columns=['kmeans_cluster', 'gmm_cluster']))

# Convert to DataFrame
pca_df = pd.DataFrame(reduced_data, columns=['PC1', 'PC2'])

# Add cluster labels to PCA DataFrame
pca_df['hierarchical_cluster'] = combined['hierarchical_cluster'].values
